In [6]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

X_mean, X_std =[0.0, 1.0]
n_inputs = 8

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std,  y

preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

2.0.0-beta1
2.2.4-tf


(<tf.Tensor: id=24, shape=(8,), dtype=float32, numpy=
 array([   4.2083,   44.    ,    5.3232,    0.9171,  846.    ,    2.337 ,
          37.47  , -122.2   ], dtype=float32)>,
 <tf.Tensor: id=20, shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextlineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)